Importando as Dependências

In [37]:
import os
import copy
# os.chdir('corpora')

from scripts.anntools import Collection
from pathlib import Path

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thiagocastroferreira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Leitura de Arquivo

In [38]:
c = Collection()

for fname in Path("original/training/").rglob("*.txt"):
  c.load(fname)

Acesso a uma instância anotada

In [39]:
c.sentences[0]

Sentence(text='La presencia del gen de células falciformes y otro normal se denomina rasgo drepanocítico.', keyphrases=[Keyphrase(text='presencia', label='Action', id=1, attr=[]), Keyphrase(text='gen', label='Concept', id=2, attr=[]), Keyphrase(text='gen de células falciformes', label='Concept', id=3, attr=[]), Keyphrase(text='normal', label='Concept', id=4, attr=[]), Keyphrase(text='rasgo drepanocítico', label='Concept', id=5, attr=[])], relations=[Relation(from='gen', to='normal', label='in-context'), Relation(from='presencia', to='gen de células falciformes', label='subject'), Relation(from='presencia', to='rasgo drepanocítico', label='same-as'), Relation(from='presencia', to='gen', label='subject')])

Acesso ao texto de uma instância

In [40]:
c.sentences[0].text

'La presencia del gen de células falciformes y otro normal se denomina rasgo drepanocítico.'

Acesso às entidades nomeadas de uma instância

In [41]:
c.sentences[0].keyphrases

[Keyphrase(text='presencia', label='Action', id=1, attr=[]),
 Keyphrase(text='gen', label='Concept', id=2, attr=[]),
 Keyphrase(text='gen de células falciformes', label='Concept', id=3, attr=[]),
 Keyphrase(text='normal', label='Concept', id=4, attr=[]),
 Keyphrase(text='rasgo drepanocítico', label='Concept', id=5, attr=[])]

Acesso às relações anotadas de uma instância

In [42]:
c.sentences[0].relations

[Relation(from='gen', to='normal', label='in-context'),
 Relation(from='presencia', to='gen de células falciformes', label='subject'),
 Relation(from='presencia', to='rasgo drepanocítico', label='same-as'),
 Relation(from='presencia', to='gen', label='subject')]

Pré-processando os Dados

In [43]:
def extract_keyphrases(keyphrases, text):
    tags = {}
    for keyphrase in sorted(keyphrases, key=lambda x: len(x.text)):
        ktext = keyphrase.text
        ktokens = [text[s[0]:s[1]] for s in keyphrase.spans]

        # casos contínuos
        idxs, ponteiro = [], 0
        for i, token in enumerate(tokens):
            if token == ktokens[ponteiro]:
                idxs.append(i)
                ponteiro += 1
            else:
                idxs, ponteiro = [], 0

            if ponteiro == len(ktokens):
                break
                
        if len(ktokens) != len(idxs):
            idxs, ponteiro = [], 0
            for i, token in enumerate(tokens):
                if token == ktokens[ponteiro]:
                    idxs.append(i)
                    ponteiro += 1

                if ponteiro == len(ktokens):
                    break

        error = False
        if len(ktokens) != len(idxs):
            error = True

        tags[keyphrase.id] = {
            'text': ktext,
            'idxs': idxs,
            'tokens': [text[s[0]:s[1]] for s in keyphrase.spans],
            'attributes': [attr.__repr__() for attr in keyphrase.attributes],
            'spans': keyphrase.spans,
            'label': keyphrase.label,
            'id': keyphrase.id,
            'error': error
        }
    return tags

data = []
for instance in c.sentences:
    text = instance.text
    tokens = nltk.word_tokenize(text.replace('–', ' – '), language='spanish')

    keyphrases = extract_keyphrases(instance.keyphrases, text)

    relations = []
    for relation in instance.relations:
        relations.append({ 
            'arg1': relation.origin,
            'arg2': relation.destination,
            'label': relation.label 
          })

    data.append({
      'text': text,
      'tokens': tokens,
      'keyphrases': keyphrases,
      'relations': relations
    })

Separando dados e salvando

In [44]:
from random import shuffle

shuffle(data)

In [45]:
size = int(len(data)*0.2)
trainset, _set = data[size:], data[:size]

size = int(len(_set)*0.5)
devset, testset = _set[size:], _set[:size]

In [46]:
import json

if not os.path.exists('preprocessed'):
    os.mkdir('preprocessed')

json.dump(trainset, open('preprocessed/trainset.json', 'w'), sort_keys=True, indent=4, separators=(',', ':'))
json.dump(devset, open('preprocessed/devset.json', 'w'), sort_keys=True, indent=4, separators=(',', ':'))
json.dump(testset, open('preprocessed/testset.json', 'w'), sort_keys=True, indent=4, separators=(',', ':'))

In [47]:
for row in trainset:
    keyphrases = row['keyphrases']
    for kid in keyphrases:
        keyphrase = keyphrases[kid]
        for i, idx in enumerate(keyphrase['idxs']):
            if i > 0:
                if keyphrase['idxs'][i-1]+1 != idx:
                    print(keyphrase)
                    print(row['tokens'])
                    print()
                    break
                    

{'text': 'exámenes de laboratorio', 'idxs': [17, 21, 22], 'tokens': ['exámenes', 'de', 'laboratorio'], 'attributes': [], 'spans': [(106, 114), (129, 131), (132, 143)], 'label': 'Concept', 'id': 2155, 'error': False}
['El', 'doctor', 'diagnosticará', 'la', 'endocarditis', 'basándose', 'en', 'sus', 'factores', 'de', 'riesgo', ',', 'historia', 'clínica', ',', 'síntomas', 'y', 'exámenes', 'del', 'corazón', 'y', 'de', 'laboratorio', '.']

{'text': 'agencia de inteligencia', 'idxs': [24, 26, 27], 'tokens': ['agencia', 'de', 'inteligencia'], 'attributes': [], 'spans': [(168, 175), (183, 185), (186, 198)], 'label': 'Concept', 'id': 3135, 'error': False}
['Los', 'talibanes', 'rápidamente', 'se', 'atribuyeron', 'responsabilidad', 'por', 'la', 'explosión', ',', 'afirmando', 'que', 'el', 'atacante', 'tenía', 'como', 'objetivo', 'dos', 'minibuses', 'que', 'transportaban', 'personal', 'de', 'la', 'agencia', 'afgana', 'de', 'inteligencia', 'y', 'que', '38', 'de', 'ellos', 'fueron', 'asesinados', '.']